In [1]:
# import dependencies
import pandas as pd
import pyodbc
import psycopg2

# define a connection string to Access database
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\23470\OneDrive\Desktop\10Alytics\capstone_project\project3\WPI.accdb;'
    )
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# retrieve table names from the Access database
table_names = [table.table_name for table in cursor.tables(tableType='TABLE')]

# save tables to csv file on local machine
for table_name in table_names:
    df = pd.read_sql_query(f'SELECT * FROM [{table_name}]', conn)
    df.to_csv(f'{table_name.replace("/", "_")}.csv', index=False)
    
# closs connection 
cursor.close()
conn.close()

In [18]:
# define a connect to postgres sever 
def get_db_connection(host, database, username, password):
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=username,
        password=password
    )
    
    return conn

# initialize connection function parameters
host = "localhost"
database = "World_Port_index"
username = "postgres"
password = "*******"

# establish a connect
con = get_db_connection(host, database, username, password)
# create cursor
curr = con.cursor()

In [3]:
# create table name wpi_data has already been created in postgres server 
create_statement = '''CREATE TABLE "Wpi_Data" (
        "World_port_index_number" DECIMAL NOT NULL,
        "Region_index" DECIMAL NOT NULL,
        "Main_port_name" VARCHAR NOT NULL,
        "Wpi_country_code" VARCHAR,
        "Latitude_degrees" DECIMAL NOT NULL,
        "Latitude_minutes" DECIMAL NOT NULL,
        "Latitude_hemisphere" VARCHAR,
        "Longitude_degrees" DECIMAL NOT NULL,
        "Longitude_minutes" DECIMAL NOT NULL,
        "Longitude_hemisphere" VARCHAR,
        "Publication" VARCHAR,
        "Chart" VARCHAR,
        "Harbor_size_code" VARCHAR,
        "Harbor_type_code" VARCHAR,
        "Shelter_afforded_code" VARCHAR,
        "Entrance_restriction_tide" BOOLEAN,
        "Entrance_restriction_swell" BOOLEAN,
        "Entrance_restriction_ice" BOOLEAN,
        "Entrance_restriction_other" BOOLEAN,
        "Overhead_limits" BOOLEAN,
        "Channel_depth" VARCHAR,
        "Anchorage_depth" VARCHAR,
        "Cargo_pier_depth" VARCHAR,
        "Oil_terminal_depth" VARCHAR,
        "Tide" DECIMAL NOT NULL,
        "Maxsize_vessel_code" VARCHAR,
        "Good_holding_ground" BOOLEAN,
        "Turning_area" BOOLEAN,
        "First_port_of_entry" BOOLEAN,
        "Us_representative" BOOLEAN,
        "Eta_message" BOOLEAN,
        "Pilotage_compulsory" BOOLEAN,
        "Pilotage_available" BOOLEAN,
        "Pilotage_local_assist" BOOLEAN,
        "Pilotage_advisable" BOOLEAN,
        "Tugs_salvage" BOOLEAN,
        "Tugs_assist" BOOLEAN,
        "Quarantine_pratique" BOOLEAN,
        "Quarantine_deratt_cert" BOOLEAN,
        "Quarantine_other" BOOLEAN,
        "Communications_telephone" BOOLEAN,
        "Communications_telegraph" BOOLEAN,
        "Communications_radio" BOOLEAN,
        "Communications_radio_tel" BOOLEAN,
        "Communications_air" BOOLEAN,
        "Communications_rail" BOOLEAN,
        "Load_offload_wharves" BOOLEAN,
        "Load_offload_anchor" BOOLEAN,
        "Load_offload_med_moor" BOOLEAN,
        "Load_offload_beach_moor" BOOLEAN,
        "Load_offload_ice_moor" BOOLEAN,
        "Medical_facilities" BOOLEAN,
        "Garbage_disposal" BOOLEAN,
        "Degauss" BOOLEAN,
        "Dirty_ballast" BOOLEAN,
        "Cranes_fixed" BOOLEAN,
        "Cranes_mobile" BOOLEAN,
        "Cranes_floating" BOOLEAN,
        "Lifts_100_tons_plus" BOOLEAN,
        "Lifts_50_100_tons" BOOLEAN,
        "Lifts_25_49_tons" BOOLEAN,
        "Lifts_0_24_tons" BOOLEAN,
        "Services_longshore" BOOLEAN,
        "Services_elect" BOOLEAN,
        "Services_steam" BOOLEAN,
        "Services_navig_equip" BOOLEAN,
        "Services_elect_repair" BOOLEAN,
        "Supplies_provisions" BOOLEAN,
        "Supplies_water" BOOLEAN,
        "Supplies_fuel_oil" BOOLEAN,
        "Supplies_diesel_oil" BOOLEAN,
        "Supplies_deck" BOOLEAN,
        "Supplies_engine" BOOLEAN,
        "Repair_code" VARCHAR,
        "Drydock" VARCHAR,
        "Railway" VARCHAR
)''';

curr.execute(create_statement)
con.commit()


In [17]:
# upload the dataframe to postgress database
# NB: a database called World_Port_index has already been created in postgres server 
with open('Wpi Data.csv', 'r') as file:
    next(file)  # Skip the header row if needed

    # Use the COPY statement to upload the data
    curr.copy_from(file, 'wpi_data', sep=',')
    con.commit()
    
# close connection   
curr.close()
con.close()